# Taller 2 · Muestreo, señales discretas y aliasing

**Asignatura:** Teoría de la Información y Procesado de Señal
**Grado en Ciencia e Ingeniería de Datos (GCED) — Universidad de A Coruña**
**Duración:** 2 horas
**Modalidad:** Jupyter Notebook con asistencia de IA

---

## Objetivos de aprendizaje

Al finalizar este taller serás capaz de:

1. Comprender el **muestreo** como discretización de señales continuas.
2. Crear una función de **consulta temporal** que traduzca instantes de tiempo a muestras discretas.
3. Comprobar empíricamente que una sinusoide muestreada **puede perder su periodicidad**.
4. Demostrar que las **frecuencias de una señal discreta están acotadas**.
5. Distinguir entre **muestreo** y **diezmado**.
6. **Observar y escuchar el aliasing** cuando se viola el criterio de Nyquist.

---

## 🎯 Reto central del taller

> **¿Qué se pierde al pasar del mundo continuo al discreto?**
>
> Al muestrear, los índices $n$ pierden la referencia temporal, las sinusoides pueden dejar de ser periódicas, y las frecuencias se "pliegan". En este taller descubrirás estas consecuencias de forma práctica.

---

## Metodología de trabajo con IA

| Puedes pedir a la IA | NO debes pedir a la IA |
|---------------------|------------------------|
| Código de generación de señales | Que interprete los resultados por ti |
| Sintaxis de funciones NumPy | Que diseñe la función de consulta temporal |
| Ayuda con gráficas (stem, plot) | Que explique por qué se pierde la periodicidad |

> *La IA te ayuda a escribir código, pero no a entender señales. Eso es tu trabajo.*

---

## Identificación del estudiante

Completa los siguientes campos con tu información personal:

- **Apellidos:** _____

- **Nombre:** _____

- **Email UDC:** _____

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from fractions import Fraction

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 11

print("✓ Entorno listo")

---

## Parte 1: Muestreo correcto — señal de referencia

### Contexto teórico

El **Teorema de Nyquist** establece que para reconstruir una señal sin ambigüedad:

$$F_s > 2 \cdot F_{\max}$$

donde $F_{\max}$ es la frecuencia más alta presente en la señal.

**Frecuencia de Nyquist:** $F_N = F_s / 2$

Al muestrear una señal continua $x(t)$ con periodo de muestreo $T_s = 1/F_s$:

$$x(t_n) = x(n \cdot T_s) \Rightarrow x[n]$$

**Importante:** $x[n]$ es una secuencia de números. El índice $n$ es un **entero**, no tiene unidades de tiempo.

---

### Bloque 1 · Generación de señal muestreada correctamente

#### 📝 Hipótesis previa

**Pregunta:** Con $F = 5$ Hz y $F_s = 50$ Hz, ¿cuántas muestras habrá por ciclo?

*Tu predicción:* `_______________`

In [ ]:
# === PARÁMETROS ===
F = 5       # Hz (frecuencia de la señal)
Fs = 50     # Hz (frecuencia de muestreo)
T = 1       # segundo
A = 1       # amplitud

# TODO: Calcula N y genera la secuencia de índices, n 

# TODO: Genera la señal sinusoidal, x

In [ ]:
# === VALIDACIÓN ===
assert x is not None, "Genera la señal"
assert len(x) == Fs * T, F"Esperado {Fs*T} muestras"

muestras_por_ciclo = Fs / F
ciclos_totales = F * T

print(f"✓ Señal generada: {len(x)} muestras")
print(f"  Muestras por ciclo: {muestras_por_ciclo}")
print(f"  Ciclos totales en {T}s: {ciclos_totales}")
print(f"  ¿Cumple Nyquist? Fs={Fs} > 2*F={2*F}: {Fs > 2*F}")

In [ ]:
# === VISUALIZACIÓN ===
plt.figure(figsize=(12, 4))
plt.stem(n[:30], x[:30], basefmt=' ')
plt.title(f'Señal muestreada correctamente: F={F} Hz, Fs={Fs} Hz')
plt.xlabel('n (muestras)')
plt.ylabel('x[n]')
plt.show()

### ✍️ Explicación (OBLIGATORIA)

**1. ¿Cuántas muestras hay por cada ciclo? ¿Es suficiente para "ver" bien la forma de onda?**

*Tu respuesta:*

```
[Escribe aquí]
```

**2. Observa que el eje X dice "n (muestras)", no "tiempo (s)". ¿Qué información necesitas para saber en qué instante de tiempo cae cada muestra?**

*Tu respuesta:*

```
[Escribe aquí]
```

---

## Parte 2: Consulta temporal — acceso a señales discretas por tiempo

### Contexto

Imagina que recibes una señal digital de un sensor de temperatura:

$$x[n] = [19.60, \; 19.68, \; 19.74, \; 19.83, \; 19.91, \; 20.01]$$

Te preguntan: **¿cuál es la temperatura en el segundo 0.75?**

Para responder, necesitas saber:
- **¿Cada cuánto tiempo se toman las muestras?** → Periodo de muestreo $T_s$ (o frecuencia $F_s$)
- **¿Cuándo se tomó la primera muestra?** → Instante inicial $t_{inicio}$

Sin estos dos datos, $x[n]$ es solo una lista de números sin referencia temporal.

**Ejemplo de la diapositiva de teoría:**
- $T_s = 0.5$ s ($F_s = 2$ Hz), $t_{inicio} = -1$ s
- La primera muestra ($n=0$ en Python) corresponde a $t = -1$ s
- Cada muestra posterior corresponde a un instante $0.5$ s más tarde

---

### Bloque 2A · Analizar qué necesita la función

#### 📝 Hipótesis previa

**Pregunta:** Si tienes `x = [19.60, 19.68, 19.74, 19.83, 19.91, 20.01]`, $F_s = 2$ Hz y $t_{inicio} = -1$ s:

- ¿A qué instante corresponde `x[0]`? ____
- ¿A qué instante corresponde `x[3]`? ____
- ¿Qué valor tiene la señal en $t = 0$ s? ____
- ¿Qué valor tiene la señal en $t = 0.75$ s? ____

In [ ]:
# === EJEMPLO: sensor de temperatura ===
x_temp = np.array([19.60, 19.68, 19.74, 19.83, 19.91, 20.01])
Fs_temp = 2       # Hz (una muestra cada 0.5 s)
t_inicio = -1.0   # la primera muestra se tomó en t = -1 s

# Calcula el vector de tiempos correspondiente a cada muestra
Ts_temp = 1 / Fs_temp
t_temp = t_inicio + np.arange(len(x_temp)) * Ts_temp

print("Muestras y sus instantes de tiempo:")
print("-" * 40)
for i, (ti, xi) in enumerate(zip(t_temp, x_temp)):
    print(f"  x[{i}] → t = {ti:+.1f} s → {xi:.2f}°")
print(f"\nPregunta: ¿Qué temperatura hay en t = 0 s?")
print(f"  n = (t - t_inicio) / Ts = (0 - ({t_inicio})) / {Ts_temp} = {(0 - t_inicio) / Ts_temp}")
print(f"  ¡Es un entero! Hay muestra exacta en t = 0 s")
print(f"\nPregunta: ¿Qué temperatura hay en t = 0.75 s?")
print(f"  n = (t - t_inicio) / Ts = (0.75 - ({t_inicio})) / {Ts_temp} = {(0.75 - t_inicio) / Ts_temp}")
print(f"  ¡No es un entero! No hay muestra exacta en t = 0.75 s")

---

### Bloque 2B · Implementar la función de consulta temporal

Crea una función `consultar_valor(x, t, Fs, t_inicio)` que:

1. Calcule el índice $n$ correspondiente al instante $t$
2. Si $n$ corresponde a una muestra exacta: devuelva ese valor y una indicación que ese valor existe (True)
3. Si $n$ NO es entero: devuelva la **media** de la muestra anterior y la siguiente, e indique que ese valor no existe (False), y, por lo tanto, lo devuelto es una **aproximación por interpolación**.

La función debe devolver una tupla `(valor, es_exacto)` donde:
- `valor` es el valor de la señal (exacto o interpolado)
- `es_exacto` es `True` si la muestra existía, `False` si fue interpolada

In [ ]:
# === IMPLEMENTACIÓN ===
def consultar_valor(x, t, Fs, t_inicio):
    """
    Consulta el valor de una señal discreta en un instante de tiempo.

    Parámetros:
        x        : np.array, señal discreta
        t        : float, instante de tiempo a consultar (en segundos)
        Fs       : float, frecuencia de muestreo (Hz)
        t_inicio : float, instante temporal de la primera muestra (s)

    Retorna:
        (valor, es_exacto) : tupla con el valor y un booleano indicando si es exacto o interpolado
    """
    Ts = 1 / Fs

    # TODO: Calcula el índice (real, no necesariamente entero), n_real
    

    # TODO: Obten el booleano es_entero con la comprobación de si n_real es un entero (True) o no (False)
    

    if es_entero:
        # TODO: Devuelve el valor exacto de la muestra correspondiente a n_entero, junto con True
        
    else:
        # TODO: Devuelve la media entre muestra del índice entero anterior y el índice entero siguiente
        

In [ ]:
# === VALIDACIÓN ===
# Test 1: consulta en instante exacto
val, exacto = consultar_valor(x_temp, 0.5, Fs_temp, t_inicio)
assert exacto == True, "t=0.5s debería corresponder a una muestra exacta"
assert np.isclose(val, 19.83), f"Valor incorrecto en t=0.5s: {val}"
print(f"✓ t=0.5s → x = {val:.2f}° ({'exacto' if exacto else 'interpolado'})")

# Test 2: consulta en instante no existente
val, exacto = consultar_valor(x_temp, 0.75, Fs_temp, t_inicio)
assert exacto == False, "t=0.75s NO corresponde a una muestra exacta"
esperado = (19.83 + 19.91) / 2  # media entre muestras adyacentes
assert np.isclose(val, esperado), f"Interpolación incorrecta: {val} != {esperado}"
print(f"✓ t=0.75s → x ≈ {val:.2f}° ({'exacto' if exacto else 'interpolado'})")

# Test 3: primera y última muestra
val, exacto = consultar_valor(x_temp, -1.0, Fs_temp, t_inicio)
assert exacto and np.isclose(val, 19.60)
print(f"✓ t=-1.0s → x = {val:.2f}° ({'exacto' if exacto else 'interpolado'})")

val, exacto = consultar_valor(x_temp, 1.5, Fs_temp, t_inicio)
assert exacto and np.isclose(val, 20.01)
print(f"✓ t=1.5s → x = {val:.2f}° ({'exacto' if exacto else 'interpolado'})")

print("\n✓ Todos los tests pasados")

In [ ]:
# === DEMOSTRACIÓN CON LA SEÑAL DE LA PARTE 1 ===
# Usamos la señal sinusoidal de la Parte 1
t_inicio_senal = 0.0  # comenzamos en t=0

# Consultas en instantes exactos e interpolados
instantes_consulta = [0.0, 0.01, 0.025, 0.05, 0.075, 0.1]

print(f"Señal: f={F} Hz, Fs={Fs} Hz, t_inicio={t_inicio_senal} s")
print("-" * 60)
for t_q in instantes_consulta:
    val, exacto = consultar_valor(x, t_q, Fs, t_inicio_senal)
    tipo = "EXACTO" if exacto else "INTERPOLADO"
    print(f"  t = {t_q:.3f} s → x ≈ {val:.4f}  [{tipo}]")

### ✍️ Explicación (OBLIGATORIA)

**1. ¿Por qué la función necesita recibir $F_s$ y $t_{inicio}$ además de la señal y el tiempo? ¿Qué pasaría si no los tuvieras?**

*Tu respuesta:*

```
[Escribe aquí]
```

**2. ¿Es la interpolación lineal (media de muestras adyacentes) una buena aproximación? ¿Cuándo podría fallar?**

*Tu respuesta:*

```
[Escribe aquí]
```

**3. Si dos sensores muestrean la misma magnitud pero con diferente $F_s$ y $t_{inicio}$, ¿pueden producir la misma secuencia $x[n]$?**

*Tu respuesta:*

```
[Escribe aquí]
```

**4. Si se pide el valor en un instante anterior a la primera muestra, o posterior a la última. ¿Cómo lo resolverías?**

*Tu respuesta:*

```
[Escribe aquí]
```

---

## Parte 3: Periodicidad de señales muestreadas

### Contexto teórico

Una señal continua $x(t) = \cos(2\pi F_0 t)$ es **siempre periódica** con periodo $T_0 = 1/F_0$.

Pero al muestrearla obtenemos $x[n] = \cos(2\pi f_0 n)$, donde $f_0 = F_0/F_s$ es la **frecuencia normalizada** (en ciclos/muestra).

La señal discreta es periódica ($x[n+N] = x[n]$ para todo $n$) **si y solo si** existe un entero $N$ tal que:

$$2\pi f_0 N = 2\pi k \quad \Rightarrow \quad f_0 = \frac{k}{N}, \quad k, N \in \mathbb{Z}$$

Es decir, **$f_0$ debe ser un número racional**.

Si $f_0$ es irracional (por ejemplo $f_0 = 1/\sqrt{2}$), la señal discreta **nunca se repite exactamente**.

---

### Bloque 3 · Caso periódico vs no periódico

#### 📝 Hipótesis previa

**Pregunta:** Sea $F_0 = 5$ Hz y $F_s = 40$ Hz. La frecuencia normalizada es $f_0 = 5/40 = 1/8$.

- ¿Es $f_0 = 1/8$ un número racional? ____
- ¿Cuál sería el periodo fundamental $N$? ____
- Si ahora $f_0 = 1/\sqrt{6} \approx 0.4082$, ¿será periódica? ____

In [ ]:
# === CASO 1: Frecuencia racional → periódica ===
f0_racional = Fraction(1, 8)  # f0 = 1/8 (racional)
N_periodo = f0_racional.denominator  # periodo fundamental
k = f0_racional.numerator

print(f"=== CASO 1: f0 = {f0_racional} (racional) ===")
print(f"  Periodo fundamental N = {N_periodo}")
print(f"  Número de ciclos por periodo k = {k}")

# Generamos señal con suficientes muestras para ver varios periodos
n_test = np.arange(40)
x_periodica = np.cos(2 * np.pi * float(f0_racional) * n_test)

# TODO: Completa la verificación empírica de que x[n] == x[n+N] para las primeras N muestras, mostrando los valores de x[n] y x[n+N], y la diferencia entre ellos
# Formato de salida sugerido:  x[1] = 0.aaaa,  x[9] = 0.bbb,  diferencia = 0.cccc
print(f"\nComprobación: ¿x[n] == x[n + {N_periodo}]?")
for i in range(N_periodo):

    

In [ ]:
# === CASO 2: Frecuencia irracional → NO periódica ===
f0_irracional = 1 / np.sqrt(70)

print(f"=== CASO 2: f0 = 1/√70 ≈ 1/{np.sqrt(70):.5f} ≈ {f0_irracional:.5f} (irracional) => El periodo sería {1/f0_irracional:.5f} y NO es entero ===")
N_aprox = int(np.round(1 / f0_irracional))
print(f"  Aproximación para considerar N entero: N_aprox = {N_aprox}")
print(f"  Número de ciclos por periodo k = {k}")

# Generamos señal con suficientes muestras para ver varios periodos
n_test2 = np.arange(40)
x_no_periodica = np.cos(2 * np.pi * float(f0_irracional) * n_test)

# TODO: Completa la verificación empírica de que x[n] == x[n+N] para las primeras N muestras, mostrando los valores de x[n] y x[n+N], y la diferencia entre ellos
# Formato de salida sugerido:  x[1] = 0.aaaa,  x[9] = 0.bbb,  diferencia = 0.cccc
print(f"\nComprobación: ¿x[n] == x[n + {N_aprox}]?")
for i in range(N_aprox):

    

In [ ]:
# === VISUALIZACIÓN COMPARATIVA ===
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

# Caso periódico
axes[0].stem(n_test[:32], x_periodica[:32], basefmt=' ', markerfmt='bo', linefmt='b-')
# Marcar los periodos
for p in range(4):
    axes[0].axvspan(p*N_periodo, (p+1)*N_periodo - 0.5, alpha=0.1, color=['blue','green','red','orange'][p])
axes[0].set_title(f'PERIÓDICA: f₀ = {f0_racional} → N = {N_periodo} (los bloques de color se repiten)')
axes[0].set_xlabel('n')
axes[0].set_ylabel('x[n]')

# Caso no periódico
axes[1].stem(n_test2[:32], x_no_periodica[:32], basefmt=' ', markerfmt='ro', linefmt='r-')
# Marcar los periodos
for p in range(4):
    axes[1].axvspan(p*N_periodo, (p+1)*N_periodo - 0.5, alpha=0.1, color=['blue','green','red','orange'][p])
axes[1].set_title(f'NO PERIÓDICA: f₀ = 1/√70 ≈ 1/{np.sqrt(70):.5f} ≈ {f0_irracional:.5f} (nunca se repite exactamente)')
axes[1].set_xlabel('n')
axes[1].set_ylabel('x[n]')

plt.tight_layout()
plt.show()

### ✍️ Explicación (OBLIGATORIA)

**1. ¿Por qué una señal continua periódica puede dejar de ser periódica al muestrearla?**

*Tu respuesta:*

```
[Escribe aquí]
```

**2. ¿Qué condición debe cumplir la relación $F_0 / F_s$ para que la señal discreta sea periódica?**

*Tu respuesta:*

```
[Escribe aquí]
```

---

## Parte 4: Frecuencias acotadas en señales discretas

### Contexto teórico

En señales continuas, la frecuencia puede crecer sin límite: $\cos(2\pi F_0 t)$ con $F_0 \to \infty$ oscila cada vez más rápido.

En señales discretas, esto **NO** es así. Observa:

$$\cos(2\pi f_0 n) = \cos\big((2\pi f_0 + 2\pi)n\big) = \cos(2\pi(f_0 + 1) \cdot n)$$

Esto es porque $n$ es entero, y sumar $2\pi n$ al argumento de un coseno no cambia nada (son vueltas completas a la circunferencia).

**Consecuencia:** Las frecuencias $f_0$ y $f_0 + 1$ producen **exactamente la misma señal discreta**.

Por tanto, todas las frecuencias distintas de una señal discreta están en el rango:

$$-\frac{1}{2} \leq f_0 < \frac{1}{2}$$

La frecuencia $f_0 = 0.5$ (media muestra por ciclo) es la **máxima frecuencia representable**, y corresponde a la alternancia $+1, -1, +1, -1, \ldots$

---

### Bloque 4A · Señales con frecuencia creciente

#### 📝 Hipótesis previa

**Pregunta:** Si generas $\cos(2\pi f_0 n)$ para $f_0 = 0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9$:

- ¿Cuáles crees que "oscilarán cada vez más rápido"? ____
- ¿Qué pasará cuando $f_0 > 0.5$? ____

In [ ]:
# === IMPLEMENTACIÓN ===
frecuencias_f0 = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1]
n_demo = np.arange(40)

fig, axes = plt.subplots(len(frecuencias_f0), 1, figsize=(14, 2.2*len(frecuencias_f0)))

for i, f0 in enumerate(frecuencias_f0):
    x_f0 = np.cos(2 * np.pi * f0 * n_demo)
    axes[i].stem(n_demo, x_f0, basefmt=' ', markerfmt='o', linefmt='-')

    # Indica la frecuencia "equivalente" en [-0.5, 0.5)
    f0_equiv = f0 - round(f0)  # reduce a [-0.5, 0.5)
    if abs(f0_equiv) < 1e-10:
        f0_equiv = 0.0
    nota = ""
    if abs(f0 - abs(f0_equiv)) > 0.001:
        nota = f" = misma señal que f₀ = {abs(f0_equiv):.2f}"

    axes[i].set_title(f'f₀ = {f0}{nota}', fontsize=10, loc='left')
    axes[i].set_ylabel('x[n]', fontsize=8)
    axes[i].set_ylim([-1.3, 1.3])

axes[-1].set_xlabel('n')
plt.suptitle('Frecuencias crecientes en señales discretas', fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# === COMPROBACIÓN NUMÉRICA ===
# Verificar que f0 y f0+1 producen la misma señal
n_check = np.arange(20)

pares_equivalentes = [(0, 1), (0.1, 1.1), (0.2, 1.2), (0.3, 0.7), (0.1, 0.9)]

print("Verificación: ¿cos(2π·f_a·n) == cos(2π·f_b·n)?")
print("=" * 50)
# TODO: Completa la verificación numérica de que x_a = np.cos(2 * np.pi * f_a * n_check) y x_b = np.cos(2 * np.pi * f_b * n_check) son iguales para cada par de frecuencias equivalentes, mostrando el valor máximo del valor absoluto de diferencia entre sus muestras
# Formato de salida sugerido: f_a vs f_b: diferencia absolutamáxima = 0.0000
for f_a, f_b in pares_equivalentes:

    

---

### Bloque 4B · ¿Qué ocurre en términos de $F_0$ y $F_s$?

Lo anterior en frecuencias normalizadas ($f_0$) se traduce a frecuencias físicas ($F_0$ en Hz):

$$f_0 = \frac{F_0}{F_s} \quad \Rightarrow \quad f_0 < \frac{1}{2} \quad \Leftrightarrow \quad F_0 < \frac{F_s}{2}$$

Es decir, la máxima frecuencia "real" que una señal discreta puede representar es la frecuencia de Nyquist $F_N=F_s/2$ (**¡el límite de Nyquist!**)

In [ ]:
# === DEMOSTRACIÓN con frecuencias físicas ===
Fs_demo = 100  # Hz de muestreo

# Señales con frecuencias crecientes, incluyendo mayores que Fs/2
frecuencias_Hz = [10, 20, 30, 40, 50, 60, 70, 80, 90]
n_d = np.arange(50)

fig, axes = plt.subplots(len(frecuencias_Hz), 1, figsize=(14, 2*len(frecuencias_Hz)))

for i, F0 in enumerate(frecuencias_Hz):
    x_F0 = np.cos(2 * np.pi * F0 * n_d / Fs_demo)
    f0_norm = F0 / Fs_demo

    # Frecuencia equivalente
    f0_eq = f0_norm
    if f0_eq > 0.5:
        f0_eq = 1.0 - f0_eq
    F0_eq = f0_eq * Fs_demo

    color = 'b' if F0 <= Fs_demo/2 else 'r'
    axes[i].stem(n_d, x_F0, basefmt=' ', markerfmt=f'{color}o', linefmt=f'{color}-')

    nota = ""
    if F0 > Fs_demo/2:
        nota = f" → PARECE {F0_eq:.0f} Hz"
    axes[i].set_title(f'F₀ = {F0} Hz (f₀ = {f0_norm:.2f}){nota}', fontsize=10, loc='left')
    axes[i].set_ylim([-1.3, 1.3])

axes[-1].set_xlabel('n')
plt.suptitle(f'Frecuencias crecientes con Fs = {Fs_demo} Hz (Nyquist = {Fs_demo/2} Hz)', fontsize=12, y=1.01)
plt.tight_layout()
plt.show()

print(f"Las señales en ROJO (F₀ > {Fs_demo/2} Hz) son indistinguibles de señales")
print(f"con frecuencia más baja. COMPRUÉBALO VISUALMENTE")
print(f"¡Las frecuencias discretas están acotadas!")

### ✍️ Explicación (OBLIGATORIA)

**1. Observa las gráficas: ¿qué pasa al llegar a $f_0 = 0.5$? ¿Y al superarlo?**

*Tu respuesta:*

```
[Escribe aquí]
```

**2. ¿Por qué $\cos(2\pi \cdot 0.3 \cdot n)$ y $\cos(2\pi \cdot 0.7 \cdot n)$ producen la misma señal discreta?**

*Tu respuesta:*

```
[Escribe aquí]
```

**3. Conecta este resultado con el límite de Nyquist: ¿por qué necesitamos $F_s > 2F_0$?**

*Tu respuesta:*

```
[Escribe aquí]
```

---

### Bloque 4C: Diezmado — reducción deliberada de muestras

#### Contexto teórico

| Concepto | Definición |
|----------|------------|
| **Muestreo** | Discretizar una señal continua a frecuencia $F_s$ |
| **Diezmado** | Tomar 1 de cada $D$ muestras de una señal **ya discreta** |

El diezmado por factor $D$ reduce el número de muestras de una señal y, por lo tanto, la frecuencia de muestreo efectiva:

$$F_s' = \frac{F_s}{D}$$

**Peligro:** Si $F_s' < 2F$, aparece aliasing.

---

#### 📝 Hipótesis previa

**Pregunta:** Con señal de $F=5$ Hz y $F_s=50$ Hz:

- Con $D=2$: $F_s' = $ ____ Hz. ¿Cumple Nyquist ($F_s' > 2F$)? ____
- Con $D=5$: $F_s' = $ ____ Hz. ¿Cumple Nyquist? ____
- Con $D=10$: $F_s' = $ ____ Hz. ¿Cumple Nyquist? ____


In [ ]:
# === IMPLEMENTACIÓN ===
factores_diezmado = [2, 5, 10]

# TODO: Diezma la señal para cada factor
señales_diezmadas = {}
for D in factores_diezmado:
    # TODO: Diezma la señal para cada factor D, guardando el resultado en x_d
    x_d = None  # Cambia este valor
    ###

    # Fs' = Fs / D
    Fs_d = Fs / D
    señales_diezmadas[D] = {
        'señal': x_d,
        'Fs_nueva': Fs_d,
        'cumple_nyquist': Fs_d > 2 * F
    }
    print(f"D={D}: Fs'={Fs_d:.1f} Hz, N'={len(x_d) if x_d is not None else '?'}, ¿Nyquist? {Fs_d > 2*F}")

In [ ]:
# === VISUALIZACIÓN COMPARATIVA ===
fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=False)

# Original
axes[0].stem(n[:30], x[:30], basefmt=' ', markerfmt='bo')
axes[0].set_title(f'Original: Fs={Fs} Hz (10 muestras/ciclo)')
axes[0].set_ylabel('x[n]')

# Diezmados
for i, D in enumerate(factores_diezmado):
    datos = señales_diezmadas[D]
    x_d = datos['señal']
    n_d = np.arange(len(x_d))
    color = 'g' if datos['cumple_nyquist'] else 'r'
    estado = '✓ OK' if datos['cumple_nyquist'] else '✗ ALIASING!'

    muestras_ciclo = datos['Fs_nueva'] / F
    axes[i+1].stem(n_d[:15], x_d[:15], basefmt=' ', markerfmt=f'{color}o')
    axes[i+1].set_title(f'D={D}: Fs\' ={datos["Fs_nueva"]:.1f} Hz ({muestras_ciclo:.1f} muestras/ciclo) {estado}')
    axes[i+1].set_ylabel('x_d[n]')

axes[-1].set_xlabel('n (muestras)')
plt.tight_layout()
plt.show()

### ✍️ Explicación (OBLIGATORIA)

**1. ¿Qué diferencia hay entre muestrear y diezmar?**

*Tu respuesta:*

```
[Escribe aquí]
```

**2. Con $D=10$. ¿Cuál debería ser la frecuencia de muestreo original $F_s$ para que la señal diezmada no tenga aliasing?**

*Tu respuesta:*

```
[Escribe aquí]
```

---

## Parte 5: Aliasing — violación del criterio de Nyquist

### Contexto teórico

El **aliasing** ocurre cuando una frecuencia $F > F_s/2$ se "disfraza" de una frecuencia más baja:

$$F_{\text{alias}} = |F - k \cdot F_s|$$

donde $k$ es el entero que minimiza el resultado y lo pone en el rango $[0, F_s/2]$.

En la Parte 4 vimos que las frecuencias discretas están acotadas. El aliasing es la **consecuencia práctica** directa: si intentamos representar una frecuencia mayor que $F_s/2$, "se pliega" al rango permitido.

---

### Bloque 5A · Aliasing visual

#### 📝 Hipótesis previa

**Pregunta:** Si $F = 15$ Hz y $F_s = 20$ Hz:

- Frecuencia de Nyquist: $F_s/2 = $ ____ Hz
- ¿Se viola Nyquist ($F > F_s/2$)? ____
- Frecuencia alias esperada: $|15 - 20| = $ ____ Hz

In [ ]:
# === IMPLEMENTACIÓN ===
F_real = 15      # Hz (frecuencia real)
Fs_bajo = 20     # Hz (frecuencia de muestreo - viola Nyquist!)
T_demo = 1       # segundo

# Señal con aliasing
N_bajo = int(Fs_bajo * T_demo)
n_bajo = np.arange(N_bajo)
x_alias = np.sin(2 * np.pi * F_real * n_bajo / Fs_bajo)

# Señal de referencia: bien muestreada
Fs_ref = 200     # Hz (mucho mayor que 2*15 = 30 Hz)
N_ref = int(Fs_ref * T_demo)
n_ref = np.arange(N_ref)
x_ref = np.sin(2 * np.pi * f_real * n_ref / Fs_ref)
t_ref = n_ref / Fs_ref

# TODO: Calcula frecuencia alias teórica, F_alias, según la fórmula de arriba para k=1

In [ ]:
# === VALIDACIÓN ===
assert F_alias == abs(F_real - Fs_bajo), "No has calculado correctamente la frecuencia alias teórica"

print(f"Frecuencia real: {F_real} Hz")
print(f"Frecuencia de muestreo: {Fs_bajo} Hz")
print(f"Frecuencia de Nyquist: {Fs_bajo/2} Hz")
print(f"¿Viola Nyquist? {F_real > Fs_bajo/2}")
print(f"Frecuencia alias teórica: {F_alias} Hz")

In [ ]:
# === VISUALIZACIÓN ===
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

# Referencia (bien muestreada)
axes[0].plot(t_ref, x_ref, 'b-', alpha=0.7, label=f'F={F_real} Hz (Fs={Fs_ref} Hz)')
axes[0].set_title('Señal de referencia (sin aliasing)')
axes[0].set_xlabel('Tiempo (s)')
axes[0].set_xlim([0, 0.4])
axes[0].legend()

# Con aliasing
t_bajo = n_bajo / Fs_bajo
axes[1].plot(t_ref, x_ref, 'b-', alpha=0.3, label='Señal real (15 Hz)')

x_alias_visual = np.sin(2 * np.pi * F_alias * t_ref + np.pi)
axes[1].plot(t_ref, x_alias_visual, 'r--', alpha=0.5, label=f'Alias aparente ({F_alias} Hz)')

axes[1].stem(t_bajo, x_alias, 'g', markerfmt='go', basefmt=' ',
             label=f'Muestras (Fs={Fs_bajo} Hz)')
axes[1].set_title(f'ALIASING: las muestras siguen la curva de {F_alias} Hz, no la de {F_real} Hz')
axes[1].set_xlabel('Tiempo (s)')
axes[1].set_xlim([0, 0.4])
axes[1].legend()

plt.tight_layout()
plt.show()

---

### Bloque 6B · Aliasing audible — escuchar el fenómeno

El aliasing no es solo un concepto matemático: tiene consecuencias **perceptibles**.

#### 📝 Hipótesis previa

Si generamos un tono de 3000 Hz y lo muestreamos a 5000 Hz:

- Frecuencia de Nyquist: $F_N = 5000/2 = 2500$ Hz
- ¿3000 Hz > 2500 Hz? → Frecuencia alias: $|3000 - 5000| = $ ____ Hz


In [ ]:
# === IMPLEMENTACIÓN ===
F_tono = 3000    # Hz
duracion = 1.5   # segundos

# Caso 1: Sin aliasing (Fs alta)
Fs_ok = 8000     # Hz (cumple Nyquist: 8000 > 2*3000)
N_ok = int(Fs_ok * duracion)
n_ok = np.arange(N_ok)
x_ok = 0.5 * np.sin(2 * np.pi * F_tono * n_ok / Fs_ok)

# Caso 2: Con aliasing (Fs baja)
Fs_mal = 5000    # Hz (viola Nyquist: 5000 < 2*3000)
N_mal = int(Fs_mal * duracion)
n_mal = np.arange(N_mal)
x_mal = 0.5 * np.sin(2 * np.pi * F_tono * n_mal / Fs_mal)

# Frecuencia alias
F_alias_audio = abs(F_tono - Fs_mal)

print(f"Tono original: {F_tono} Hz")
print(f"\nCaso 1 (sin aliasing):")
print(f"  Fs = {Fs_ok} Hz, Frecuencia de Nyquist F_N = {Fs_ok/2} Hz → OK")
print(f"\nCaso 2 (con aliasing):")
print(f"  Fs = {Fs_mal} Hz, Frecuencia de Nyquist F_N = {Fs_mal/2} Hz → ALIASING")
print(f"  Frecuencia alias: |{F_tono} - {Fs_mal}| = {F_alias_audio} Hz")

In [ ]:
# === REPRODUCCIÓN DE AUDIO ===
print("Escucha el tono SIN aliasing (3000 Hz):")
display(Audio(x_ok, rate=Fs_ok))

print(f"\nEscucha el tono CON aliasing (sonará como {F_alias_audio} Hz):")
display(Audio(x_mal, rate=Fs_mal))

# Tono de referencia a la frecuencia alias
Fs_ref_audio = 8000
N_ref_audio = int(Fs_ref_audio * duracion)
n_ref_audio = np.arange(N_ref_audio)
x_ref_alias = 0.5 * np.sin(2 * np.pi * F_alias_audio * n_ref_audio / Fs_ref_audio)

print(f"\nTono de referencia a {F_alias_audio} Hz (para comparar):")
display(Audio(x_ref_alias, rate=Fs_ref_audio))

### ✍️ Explicación (OBLIGATORIA)

**1. Observa la gráfica de aliasing: los puntos verdes ¿siguen la curva azul (15 Hz) o la roja (5 Hz)?**

*Tu respuesta:*

```
[Escribe aquí]
```

**2. ¿Suenan igual el segundo y tercer audio? ¿Qué frecuencia percibes en cada uno?**

*Tu respuesta:*

```
[Escribe aquí]
```

**3. Conecta el aliasing con la Parte 4: ¿por qué el aliasing es una consecuencia directa de que las frecuencias discretas están acotadas?**

*Tu respuesta:*

```
[Escribe aquí]
```

**4. Si solo tuvieras las muestras (sin saber la señal original), ¿podrías saber que hubo aliasing?**

*Tu respuesta:*

```
[Escribe aquí]
```

---

## 🔍 Checkpoint del profesor

- [ ] Señal periódica correcta con Nyquist cumplido
- [ ] Función de consulta temporal implementada con interpolación
- [ ] Periodicidad verificada: caso racional vs irracional
- [ ] Frecuencias acotadas demostradas visualmente
- [ ] Diezmado implementado y efectos visualizados
- [ ] Aliasing demostrado visual y auditivamente

---

## Preguntas de control

### P1. ¿Qué información necesitas, además de la secuencia $x[n]$, para saber qué valor tiene la señal en un instante $t$ concreto?

*Tu respuesta:*

```
[Escribe aquí]
```

---

### P2. Explica por qué $\cos(2\pi \cdot 0.3 n)$ y $\cos(2\pi \cdot 0.7 n)$ son la misma señal discreta.

*Tu respuesta:*

```
[Escribe aquí]
```

---

### P3. Sea $x(t) = \cos(2\pi \cdot 7 t)$ muestreada con $F_s = 40$ Hz. ¿Es periódica la señal discreta? Calcula el periodo $N$.

*Tu respuesta:*

```
[Muestra el cálculo]
```

---

### P4. Calcula: Si $f = 45$ Hz y $F_s = 80$ Hz, ¿cuál es la frecuencia alias?

*Tu respuesta:*

```
[Muestra el cálculo]
```

---

### P5. ¿Por qué el aliasing es irreversible?

*Tu respuesta:*

```
[Escribe aquí]
```

---

## ✅ Checklist final

- [ ] Run All sin errores
- [ ] Muestreo correcto implementado (Parte 1)
- [ ] Función de consulta temporal con interpolación (Parte 2)
- [ ] Periodicidad: caso racional vs irracional demostrado (Parte 3)
- [ ] Frecuencias acotadas visualizadas (Parte 4)
- [ ] Diezmado con diferentes factores (Parte 5)
- [ ] Aliasing visualizado en tiempo y escuchado en audio (Parte 6)
- [ ] Todas las explicaciones completadas

---

## 📚 Resumen de conceptos clave

| Concepto | Definición |
|----------|------------|
| **Frecuencia de Nyquist** | $F_N = F_s/2$ |
| **Criterio de Nyquist** | $F_s > 2 f_{\max}$ para evitar aliasing |
| **Consulta temporal** | Necesita $F_s$ y $t_{inicio}$ para traducir $t \to n$ |
| **Frecuencia normalizada** | $F_0 = F_0/F_s$ (ciclos/muestra) |
| **Periodicidad discreta** | Solo si $F_0 = k/N$ es racional |
| **Frecuencias acotadas** | $f_0 \in [-1/2, 1/2)$ — no hay frecuencias infinitas |
| **Muestreo vs Diezmado** | Continua→discreta vs reducir muestras |
| **Aliasing** | Frecuencias altas "se pliegan" al rango $[0, F_s/2]$ |
| **Irreversibilidad** | No se puede recuperar la frecuencia original |

---
